In [6]:
import random

import gym
from gym import wrappers
from gym.error import ResetNeeded

import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam


import numpy as np

In [7]:
def get_model():
    model = Sequential()
    model.add(Dense(4, input_shape=[1, 4], activation='relu', kernel_initializer='random_uniform'))
    model.add(Dense(6, activation='relu', kernel_initializer='random_uniform'))
    model.add(Dense(2, kernel_initializer='random_uniform'))
    return model

In [8]:
TRAIN_MODE = True

In [10]:
env = gym.make('CartPole-v0')
env = wrappers.Monitor(env, '/tmp/cartpole-experiment-1', force=True)


WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [69]:
CONFIDENCE_LEVEL = .9
LEARNING_RATE = .9
STEPS_IN_HIST = 3

observ_hist = []
reward_hist = []
s = env.reset()
done = False

# if TRAIN_MODE:
#     model = get_model()
#     model.compile('adam', 'mse')

max_angle = 0
pos_cicles = 0
neg_cicles = 0

while True:
    try:
#         observ_hist.append(s)
#         reward_hist.append(0 if done else 1)
        
        if TRAIN_MODE and np.random.rand() < CONFIDENCE_LEVEL:
            Q = random.choice([np.array([1, 0]).reshape((-1, 2)), 
                               np.array([0, 1]).reshape((-1, 2))])
#             print Q.shape
        else:
            Q = model.predict_proba(s.reshape((-1, 1, 4)), verbose=0)
            
        
        s1, _, done, info = env.step(Q.argmax())
#         print(s1)
#         reward = getting_reward(s1)
        reward = 0
        if s1[1] > 0:
            pos_cicles += 1
            if neg_cicles <= 2:
                neg_cicles = 0
        else:
            neg_cicles += 1
            if pos_cicles <= 2:
                pos_cicles = 0 

        if neg_cicles > 2 and pos_cicles > 2:
#             print('agent boosted for balancing attempt')
            reward += 1
#             reward_boost *= reward_boost_scaler + (10/((neg_cicles + pos_cicles)/2))
            print('here')
            pos_cicles, neg_cicles = 0, 0
        
        if TRAIN_MODE:
            Q_tmp = model.predict_proba(s1.reshape((-1, 1, 4)))

            Q_traget = Q
            action = Q.argmax()
            Q_traget[0, action] = reward + LEARNING_RATE * Q_tmp.max()
            observ_hist.append([s.reshape((-1, 4)),  Q_traget.reshape((-1, 2))])
            
            model.fit(s.reshape((-1, 1, 4)), Q_traget.reshape((-1, 1, 2)), verbose=0)

        s = s1
        
    except ResetNeeded:
        s = env.reset()

IndexError: index 1 is out of bounds for axis 1 with size 1

In [ ]:
len(observ_hist)

o_h, r_h = [], []
for observ in observ_hist:
    o_h.append(observ[0])
    r_h.append(observ[1])


In [ ]:
X=np.array(o_h)
Y=np.array(r_h)

In [ ]:
reward_hist

In [ ]:
t = np.array([10,2])

In [ ]:
t.argmax()

In [ ]:
env.reset()

In [ ]:
model = get_model()
model.compile('adam', 'mse')

In [ ]:
model.fit(X, Y, epochs=100000, batch_size=128, verbose=1)

In [ ]:
Y.shape